In [1]:
#!nvidia-smi

### Get dataset

In [1]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
main_path = "/content/gdrive/MyDrive/Chicken Health and Behavior Detection"
source_path = f'{main_path}/data/processed/images_datasets/data.zip'
destination_path = '/content'

In [6]:
!cp "$source_path" "$destination_path"

In [7]:
!unzip -q "$destination_path/data.zip" -d "$destination_path/data"

### Load and split data

In [8]:
!cp "$main_path/utils/split_dataset.py" "$destination_path"

In [9]:
!python split_dataset.py --datapath="$destination_path/data" --train_pct=0.9


Total images : 349
Train: 314 || Val: 35



Ultralytics YOLO model

In [ ]:
!pip install ultralytics

Train Configuration

In [11]:
!cp "$main_path/utils/create_data_yaml.py" "$destination_path"

In [12]:
!python create_data_yaml.py --classes="$destination_path/data/classes.txt" --output="$destination_path/data.yaml"

Run Model Training

In [13]:
!yolo detect train data="$destination_path/data.yaml" model=yolo11s.pt epochs=100 imgsz=640

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
100% 18.4M/18.4M [00:00<00:00, 170MB/s]
Ultralytics 8.3.163 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, 

In [14]:
!yolo detect predict model=runs/detect/train/weights/best.pt source=data/validation/images save=True

Ultralytics 8.3.163 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs

image 1/35 /content/data/validation/images/03400_jpg.rf.56745a1791753c04b4ebe10294035be4.jpg: 384x640 56 0s, 51.8ms
image 2/35 /content/data/validation/images/03402_jpg.rf.533de42925376f90af895eb386538d35.jpg: 384x640 56 0s, 11.1ms
image 3/35 /content/data/validation/images/03416_jpg.rf.6eb0f810f90bd242d4ecea8defdcf01f.jpg: 384x640 53 0s, 10.8ms
image 4/35 /content/data/validation/images/03452_jpg.rf.6f4678227c0f33deeace5d4cd4119105.jpg: 384x640 47 0s, 10.9ms
image 5/35 /content/data/validation/images/03462_jpg.rf.a9c6eda813c03f2f4bccc3a41f11d1de.jpg: 384x640 53 0s, 10.8ms
image 6/35 /content/data/validation/images/03465_jpg.rf.d2bfd0ddd1210ea5a6e16889e6e7f0db.jpg: 384x640 51 0s, 10.8ms
image 7/35 /content/data/validation/images/03466_jpg.rf.8967a6a1ebd07737d642eb3a50aba824.jpg: 384x640 51 0s, 10.8ms
image 8/35 /content/

In [15]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'/content/runs/detect/predict/*.jpg')[:10]:
  display(Image(filename=image_path, height=400))
  print('\n')

Output hidden; open in https://colab.research.google.com to view.

In [16]:
!mkdir -p "$destination_path/model_yolov11s" #Create output

#Copy best weights into the output folder
!cp "$destination_path/runs/detect/train/weights/best.pt" "$destination_path/model_yolov11s/yolov11s.pt"

#Copy training results folder
!cp -r "$destination_path/runs/detect/train/weights" "$destination_path/model_yolov11s/train"

#Zip into one file
!zip -r "$destination_path/model_yolov11s.zip" -j "$destination_path/model_yolov11s"

  adding: best.pt (deflated 8%)
  adding: last.pt (deflated 8%)
  adding: yolov11s.pt (deflated 8%)


In [ ]:
#from google.colab import files
#files.download('/content/model_yolov11s.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Test load model

In [17]:
import time
from ultralytics import YOLO

In [21]:
model = YOLO('model_yolov11s/yolov11s.pt')
source_ces = f"{main_path}/data/processed/video_test.mp4"

In [22]:
f_count = 0
start_time = time.time()
results = model(source_ces, stream=True, save=True)

In [23]:
for r in results:
    boxes = r.boxes
    masks = r.masks
    probs = r.probs

    f_count += 1

end_time = time.time()
total_time = end_time - start_time
fps = f_count / total_time

print(f'Total frames processed : {f_count}')
print(f'Total time taken : {total_time:.1f} seconds')
print(f'Frames per second : {fps:.1f}')


video 1/1 (frame 1/927) /content/gdrive/MyDrive/Chicken Health and Behavior Detection/data/processed/video_test.mp4: 384x640 62 0s, 61.3ms
video 1/1 (frame 2/927) /content/gdrive/MyDrive/Chicken Health and Behavior Detection/data/processed/video_test.mp4: 384x640 61 0s, 10.9ms
video 1/1 (frame 3/927) /content/gdrive/MyDrive/Chicken Health and Behavior Detection/data/processed/video_test.mp4: 384x640 61 0s, 11.3ms
video 1/1 (frame 4/927) /content/gdrive/MyDrive/Chicken Health and Behavior Detection/data/processed/video_test.mp4: 384x640 60 0s, 10.9ms
video 1/1 (frame 5/927) /content/gdrive/MyDrive/Chicken Health and Behavior Detection/data/processed/video_test.mp4: 384x640 60 0s, 12.3ms
video 1/1 (frame 6/927) /content/gdrive/MyDrive/Chicken Health and Behavior Detection/data/processed/video_test.mp4: 384x640 60 0s, 10.9ms
video 1/1 (frame 7/927) /content/gdrive/MyDrive/Chicken Health and Behavior Detection/data/processed/video_test.mp4: 384x640 60 0s, 11.6ms
video 1/1 (frame 8/927) /c